# __SMS SPAM Classification__ 
#### _Name: Priyanka M_
#### _Register Number: 311019104066_

### Import required library

In [39]:
%pip install keras_preprocessing

  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

### Read dataset and do preprocessing

In [10]:
df=pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [11]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [12]:
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [13]:
X=df.v2
Y=df.v1
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=Y.reshape(-1,1)

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [41]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

### Create Model and Add layers (LSTM ,Dense-(Hidden Layers),Ouput)

In [28]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

### Compile the model

In [29]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

### Fit the Model

In [42]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 18s 298ms/step - loss: 0.3267 - accuracy: 0.8757 - val_loss: 0.1390 - val_accuracy: 0.9557
Epoch 2/10
30/30 [==============================] - 6s 208ms/step - loss: 0.0826 - accuracy: 0.9818 - val_loss: 0.0623 - val_accuracy: 0.9842
Epoch 3/10
30/30 [==============================] - 6s 207ms/step - loss: 0.0414 - accuracy: 0.9886 - val_loss: 0.0545 - val_accuracy: 0.9852
Epoch 4/10
30/30 [==============================] - 8s 258ms/step - loss: 0.0338 - accuracy: 0.9900 - val_loss: 0.0559 - val_accuracy: 0.9863
Epoch 5/10
30/30 [==============================] - 7s 238ms/step - loss: 0.0253 - accuracy: 0.9926 - val_loss: 0.0566 - val_accuracy: 0.9863
Epoch 6/10
30/30 [==============================] - 7s 221ms/step - loss: 0.0185 - accuracy: 0.9942 - val_loss: 0.0684 - val_accuracy: 0.9842
Epoch 7/10
30/30 [==============================] - 8s 255ms/step - loss: 0.0129 - accuracy: 0.9963 - val_loss: 0.0710 - val_accuracy: 0.9852
Epoch

### Save the Model

In [43]:
model.save("model_1")

INFO:tensorflow:Assets written to: model_1\assets


INFO:tensorflow:Assets written to: model_1\assets


### Test the Model

In [44]:
test_sequences=tok.texts_to_sequences(X_test)
test_sequences_matrix =sequence.pad_sequences(test_sequences,maxlen=max_len)

In [45]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 35ms/step - loss: 0.0833 - accuracy: 0.9821
Accuracy: 0.982


In [46]:
y_pred =model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 2s 23ms/step
[[0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [1.   ]
 [0.   ]
 [0.984]
 [0.001]
 [0.   ]]


In [47]:
print(Y_test[25:40])

[[0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]]
